In [2]:
import os
import json
import random
from pathlib import Path

# Define the path to the training data
arc_data_path = Path("ARC-AGI/data/training")

# Check if the directory exists
if not arc_data_path.exists():
    print(f"Directory {arc_data_path} does not exist. Please make sure the ARC-AGI repository is cloned.")
else:
    # Get all JSON files in the training directory
    json_files = list(arc_data_path.glob("*.json"))
    
    if not json_files:
        print(f"No JSON files found in {arc_data_path}")
    else:
        # Select a random JSON file
        random_file = random.choice(json_files)
        print(f"Selected random file: {random_file}")
        
        # Load and display the JSON content
        with open(random_file, 'r') as f:
            data = json.load(f)
            print(f"File content: {json.dumps(data, indent=2)}")


Selected random file: ARC-AGI\data\training\a61ba2ce.json
File content: {
  "train": [
    {
      "input": [
        [
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        [
          0,
          0,
          0,
          0,
          0,
          0,
          8,
          8,
          0,
          0,
          0,
          0,
          0
        ],
        [
          0,
          0,
          0,
          0,
          0,
          0,
          8,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        [
          0,
          2,
          2,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        [
          0,
          0,
          2,
          0,
          0,
          0,
          0,
          0,
          0,
       

In [3]:
print(len(data["train"]))

2


In [5]:
input_1 = data["train"][0]["input"]
output_1 = data["train"][0]["output"]
input_2 = data["train"][1]["input"]
output_2 = data["train"][1]["output"]

In [6]:
prompt = f"""You are meant to write a code that maps inputs to outputs.
Input 1: {input_1}
Output 1: {output_1}

Write the code that does the transformation. Output the code within <code> </code> brackets."""

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

c:\Users\axeld\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [8]:
print(response)

```python
def transform(input_matrix):
    output_matrix = []
    for row in input_matrix:
        transformed_row = []
        for i in range(len(row)):
            if row[i] == 8:
                transformed_row.append(8)
            elif row[i] == 0:
                transformed_row.append(2)
            else:
                transformed_row.append(row[i])
        output_matrix.append(transformed_row)
    return output_matrix

input_matrix = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0], [0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 


In [9]:
def transform(input_matrix):
    output_matrix = []
    for row in input_matrix:
        transformed_row = []
        for i in range(len(row)):
            if row[i] == 8:
                transformed_row.append(8)
            elif row[i] == 0:
                transformed_row.append(2)
            else:
                transformed_row.append(row[i])
        output_matrix.append(transformed_row)
    return output_matrix

In [10]:
output_attempt = transform(input_2)
print(output_2 == output_attempt)

False
